# Kürzeste Pfade
(von einem gegebenen Knoten zu allen anderen Knoten)

## Tools für Experimente und Darstellung

Wir verwenden hier wieder `networkx` zur Repräsentation und zur Darstellung von Graphen.

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
from networkx.drawing.layout import circular_layout
import networkx as nx

Wir erstellen zufällig einen gerichteten Beispielgraphen. Zur besseren Darstellbarkeit verhindern wir die gleichzeitige Existenz von Kanten (u,v) und (v,u), dies ist für die Korrektheit der Algorithmen jedoch nicht erforderlich.

In [ ]:
import random

def create_random_weighted_digraph(no_nodes, no_edges, min_edge_weight, max_edge_weight):
    graph = nx.DiGraph()
    graph.add_nodes_from(range(no_nodes))
    all_pairs = list((x,y) for x in range(no_nodes)
                           for y in range(x + 1, no_nodes))
    # print(all_pairs)
    possible_weights = list(range(min_edge_weight, max_edge_weight + 1))
    for n1, n2 in random.sample(all_pairs, no_edges):
        weight = random.choice(possible_weights)
        v, w = random.choice(((n1, n2), (n2, n1)))
        graph.add_edge(v, w, weight=weight)
    return graph
        
graph = create_random_weighted_digraph(8, 16, 0, 30)

In [ ]:
pos = circular_layout(graph)
edge_labels = dict([((u,v),d['weight']) for u,v,d in graph.edges(data=True)])
nx.draw(graph, pos, with_labels=True, node_size=300, node_color='lightblue')
_ = nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)

 ## Dijkstras Algorithmus

Für Dijkstras Algorithmus benötigen wir eine indizierte Priority-Queue. Wir verwenden hierzu einen Wrapper um das Modul `pqdict`:

In [ ]:
from pqdict import pqdict

class IndexMinPQ:
    def __init__(self):
        self.pq = pqdict()

    def del_min(self):
        return self.pq.pop()

    def insert(self, entry, value):
        self.pq.additem(entry, value)

    def contains(self, entry):
        return entry in self.pq

    def change(self, entry, priority):
        self.pq.updateitem(entry, priority)

    def empty(self):
        return len(self.pq) == 0
    
    def entries(self):
        for x in self.pq:
            yield x

Wie bereits in früheren Fällen, verwenden wir im Notebook keine explizite Repräsentation der Kanten, sondern die API von networkx. Daher speichern wir auch kein Array `edge_to`, sondern merken uns stattdessen nur den Vorgängerknoten auf dem besten bekannten Pfad zu einem Knoten (wir bereits früher in einem Array `parent`). Da wir keine Multigraphen mit parallelen Kanten betrachten, gibt es jeweils nur eine Kante von `parent[i]` zu `i`und die Extraktion der kürzesten Pfade wird dadurch nicht komplizierter.

In [ ]:
class DijkstraSSSP:
    def __init__(self, graph, start_node, node_positions):
        self.parent = [None] * len(graph)
        self.distance = [float('inf')] * len(graph)
        pq = IndexMinPQ()

        self.distance[start_node] = 0
        pq.insert(start_node, 0)
        while not pq.empty():
            self.relax(graph, pq.del_min(), pq)
            self.dump(graph, node_positions, pq)

    def relax(self, graph, v, pq):
        print("Nach Relaxierung von Knoten %i:" % v)
        for w in graph.successors(v):
            edge_weight = \
                graph.get_edge_data(v,w)["weight"]
            if self.distance[v] + edge_weight < \
               self.distance[w]:
                self.parent[w] = v
                self.distance[w] = self.distance[v] + edge_weight
                if pq.contains(w):
                    pq.change(w, self.distance[w])
                else:
                    pq.insert(w, self.distance[w])

    def path_to(self, node):
        if self.distance[node] == float('inf'):
            return None
        elif self.parent[node] == None:
            return [node]
        else:
            path = self.path_to(self.parent[node])
            path.append(node)
            return path
            
    def dump(self, graph, node_positions, queue):
        print("distances:", self.distance)
        print("parents:  ", self.parent)
        reached_nodes = set(x for x in range(len(graph)) if self.distance[x] != float('inf'))
        open_nodes = set(x for x in queue.entries())
        finished_nodes = reached_nodes - open_nodes
        finished_tree = [(self.parent[i],i) for i in finished_nodes if self.parent[i] is not None]
        open_edges = [(self.parent[i],i) for i in open_nodes if self.parent[i] is not None]

        nx.draw(graph, pos, with_labels=True, node_size=300, node_color='lightblue', edge_color="gray")
        edge_labels=dict([((u,v,),d['weight']) for u,v,d in graph.edges(data=True)])
        nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)
        nx.draw_networkx_nodes(graph, pos, nodelist=finished_nodes, node_color='r')
        nx.draw_networkx_edges(graph, pos, edgelist=finished_tree, edge_color='r')

        nx.draw_networkx_nodes(graph, pos, nodelist=open_nodes, node_color='b')
        nx.draw_networkx_edges(graph, pos, edgelist=open_edges, edge_color='b')
        plt.show()

In [ ]:
d = DijkstraSSSP(graph, 0, pos)

In [ ]:
print(d.path_to(2))

 ## Azyklische Graphen

Wir erstellen zunächst einmal einen zufälligen azyklischen, gewichteten Graphen:

In [ ]:
from networkx.generators.random_graphs import gnp_random_graph

def random_dag(number_nodes, min_weight, max_weight):
    random_graph = gnp_random_graph(number_nodes, 0.5, directed=True)
    # filter edges to get acyclic graph
    dag = nx.DiGraph((u,v,{'weight': random.randint(min_weight, max_weight)})
                     for u,v in random_graph.edges()
                     if u < v)
    # re-insert lost nodes
    for node in range(number_nodes):
        if node not in dag:
            dag.add_node(node)                           
    return dag
    
dag = random_dag(7, 0, 10)

In [ ]:
pos_dag = circular_layout(dag)
nx.draw(dag, pos_dag, with_labels=True, node_size=300, node_color='lightblue')
edge_labels_dag = dict([((u,v),d['weight']) for u,v,d in dag.edges(data=True)])
_ = nx.draw_networkx_edge_labels(dag, pos_dag, edge_labels=edge_labels_dag)

Für azyklische Graphen benötigen wir keine Priority-Queue, sondern können stattdessen eine topologische Sortierung als geeignete Relaxierungsreihenfolge der Knoten verwenden.

In [ ]:
from networkx.algorithms.dag import topological_sort

class AcyclicSSSP:
    def __init__(self, graph, start_node, drawing_pos):
        self.parent = [None] * len(graph)
        self.distance = [float('inf')] * len(graph)
        
        processed_nodes = [] # only for drawing

        self.distance[start_node] = 0
        top_order = list(topological_sort(dag))
        print("Verwende folgende topologische Sortierung:", top_order, "\n")
        
        start_pos = top_order.index(start_node)
        for node in top_order[start_pos:]:
            self.relax(graph, node)
            processed_nodes.append(node)
            self.dump(graph, processed_nodes, drawing_pos)

    def relax(self, graph, v):
        print("Nach Relaxierung von Knoten %i:" % v)
        for w in graph.successors(v):
            edge_weight = graph.get_edge_data(v,w)["weight"]
            if self.distance[v] + edge_weight < self.distance[w]:
                self.parent[w] = v
                self.distance[w] = self.distance[v] + edge_weight

    def path_to(self, node):
        if self.distance[node] == float('inf'):
            return None
        elif self.parent[node] == None:
            return [node]
        else:
            path = self.path_to(self.parent[node])
            path.append(node)
            return path
            
    def dump(self, graph, processed_nodes, pos):
        print("distances:", self.distance)
        print("parents:  ", self.parent)
        
        nx.draw(graph, pos, with_labels=True, node_size=300, node_color='lightblue', edge_color="gray")
        edge_labels=dict([((u,v,),d['weight']) for u,v,d in graph.edges(data=True)])
        nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)
        
        reached_nodes = set(x for x in range(len(graph))
                            if self.distance[x] != float('inf') and x not in processed_nodes)
        best_edges = [(self.parent[i],i) for i in reached_nodes
                      if i not in processed_nodes and self.parent[i] is not None]
        finished_edges = [(self.parent[i],i) for i in processed_nodes
                          if self.parent[i] is not None]
        
        nx.draw_networkx_nodes(graph, pos, nodelist=processed_nodes, node_color='r')
        nx.draw_networkx_edges(graph, pos, edgelist=finished_edges, edge_color='r')
        nx.draw_networkx_nodes(graph, pos, nodelist=reached_nodes, node_color='b')
        nx.draw_networkx_edges(graph, pos, edgelist=best_edges, edge_color='b')
        plt.show()

In [ ]:
assp = AcyclicSSSP(dag, 4, pos_dag)

In [ ]:
print(assp.path_to(3))